In [38]:
import pandas as pd
# CB = pd.read_csv("전통주.csv", header=0,index_col=0, engine='python')
CB = pd.read_csv("전통주.csv", header=0, engine='python')
del CB['Unnamed: 12']
CB.columns = ['술 종류','술 이름','양조장 위치','도수','용량(ml)','가격 - 공식홈페이지 가격(1병 기준), 없으면 네이버 스토어 최저가','별점 - 점수별 개수 평균(점수*개수/전체개수)','탄산','맛','향','리뷰 빈도수(리뷰개수 가장 적은걸로 개수 통일화 후 랜덤으로 뽑기)','선물용(디자인, 병모양, 이쁨, 패키지, 설날, 크리스마스 등)']
CB.drop([0], axis=0, inplace=True)
CB.drop([3], axis=0, inplace=True)
CB = CB.reset_index(drop=True)
CB.head()

,술 종류,술 이름,양조장 위치,도수,용량(ml),"가격 - 공식홈페이지 가격(1병 기준), 없으면 네이버 스토어 최저가",별점 - 점수별 개수 평균(점수*개수/전체개수),탄산,맛,향,리뷰 빈도수(리뷰개수 가장 적은걸로 개수 통일화 후 랜덤으로 뽑기),"선물용(디자인, 병모양, 이쁨, 패키지, 설날, 크리스마스 등)"
0,막걸리,배상면주가 느린마을 막걸리,서울특별시 서초구 강남대로 27길 7-9 우곡빌딩 6층,6,1000,10000,4.8,O,"고소하다\n뽀얗다\n담백하다\n크림,우유(부드럽다)\n달맛(달다,달짝지근하다,달달하...",사과\n요구르트\n곡물\n과일,18923,"선물','추석','설날','크리스마스','집들이','기념','대접'"
1,막걸리,복순도가,울산광역시 울주군 상북면 향산동길 50,6.5,935,12000,4.8,O,산뜻하다\n스파클링(탄산)\n꿀떡꿀떡\n순하다\n달짝지근하다(달달)\n부드럽다\n고...,청포도\r\n누룩\r\n포도\r\n복숭아\r\n향기,27821,"부모님','가족','지인','친구','명절','추석'"
2,막걸리,죽향도가,전라남도 담양군 담양읍 삼다리 78,6,600,3800,4.8,X,부드럽다\r\n순하다\r\n굉장하다\r\n깔끔하다\r\n달콤하다\r\n달달\r\n중...,꿀향,5916,생일\r\n부모님\r\n가족\r\n친구\r\n명절\r\n추석
3,과실주,배상면주가 빙탄복,서울특별시 서초구 강남대로 27길 7-9 우곡빌딩 6층,7,370,4000,4.8,o,탄산음료같다\n도수가높지않다\n뒷맛이 깔끔,복분자,7754,혼술
4,과실주,산내들 소백산 스위트 와인,경상북도 영주시 단산면 좌석리 277-1,12,750,15500,4.76,x,부드러움\n목넘김\n,향긋하다,3541,"초보자용, 가격대비, 고급스러움(맛, 포장)"


In [39]:
addr=[]
for adress in CB['양조장 위치'] :
    addr.append(str(adress).split())

addr

[['서울특별시', '서초구', '강남대로', '27길', '7-9', '우곡빌딩', '6층'],
 ['울산광역시', '울주군', '상북면', '향산동길', '50'],
 ['전라남도', '담양군', '담양읍', '삼다리', '78'],
 ['서울특별시', '서초구', '강남대로', '27길', '7-9', '우곡빌딩', '6층'],
 ['경상북도', '영주시', '단산면', '좌석리', '277-1'],
 ['세종특별자치시', '전의면', '가느실길', '127'],
 ['전라북도', '무주군', '안성면', '장무로', '1375-7'],
 ['충청북도', '영동군', '영동읍', '조현길', '30'],
 ['제주특별자치도', '제주시', '애월읍', '애원로', '283'],
 ['충청북도', '충주시', '산척면', '샛강영길', '39'],
 ['충청남도', '논산시', '양촌면', '매죽헌로', '1665번길', '14-7'],
 ['경남', '창녕군', '대지면', '대지농공단지길', '40'],
 ['경기도', '파주시', '파주읍', '윗가마울길', '149'],
 ['경기도', '파주시', '파주읍', '윗가마울길', '149'],
 ['경북', '영덕군', '강구면', '소월1길', '16-10'],
 ['경북', '영덕군', '강구면', '소월1길', '16-10'],
 ['경북', '영덕군', '강구면', '소월1길', '16-10'],
 ['경북', '안동시', '풍산읍', '산업단지6길', '6'],
 ['경북', '안동시', '풍산읍', '산업단지6길', '6'],
 ['경북', '안동시', '풍산읍', '산업단지6길', '6'],
 ['서울시', '은평구', '증산로7길', '28-13'],
 ['서울시', '은평구', '증산로7길', '28-13'],
 ['경상북도', '안동시', '풍산읍', '㈜', '안동소주일품'],
 ['경상북도', '안동시', '풍산읍', '㈜', '안동소주일품'],
 ['강원', '속초시', '이목로

In [40]:
addr2=[]
for i in range(len(addr)):
    if addr[i][0] == "서울" :
        addr[i][0] = "서울특별시"
    elif addr[i][0] == "서울시" :
        addr[i][0] = "서울특별시"
    elif addr[i][0] == "부산시" :
        addr[i][0] = "부산광역시"
    elif addr[i][0] == "인천" :
        addr[i][0] = "인천광역시"
    elif addr[i][0] == "광주" :
        addr[i][0] = "광주광역시"
    elif addr[i][0] == "울산시" :
        addr[i][0] = "울산광역시"
    elif addr[i][0] == "세종시" :
        addr[i][0] = "세종특별자치시"
    elif addr[i][0] == "경기" :
        addr[i][0] = "경기도"
    elif addr[i][0] == "충북" :
        addr[i][0] = "충청북도"
    elif addr[i][0] == "충남" :
        addr[i][0] = "충청남도"
    elif addr[i][0] == "전북" :
        addr[i][0] = "전라북도"
    elif addr[i][0] == "전남" :
        addr[i][0] = "전라남도"
    elif addr[i][0] == "경북" :
        addr[i][0] = "경상북도"
    elif addr[i][0] == "경남" :
        addr[i][0] = "경상남도"
    elif addr[i][0] == "제주" :
        addr[i][0] = "제주특별자치도"
    elif addr[i][0] == "제주도" :
        addr[i][0] = "제주특별자치도"
    elif addr[i][0] == "제주시" :
        addr[i][0] = "제주특별자치도"    
    addr2.append(''.join(addr[i]))
addr2

['서울특별시서초구강남대로27길7-9우곡빌딩6층',
 '울산광역시울주군상북면향산동길50',
 '전라남도담양군담양읍삼다리78',
 '서울특별시서초구강남대로27길7-9우곡빌딩6층',
 '경상북도영주시단산면좌석리277-1',
 '세종특별자치시전의면가느실길127',
 '전라북도무주군안성면장무로1375-7',
 '충청북도영동군영동읍조현길30',
 '제주특별자치도제주시애월읍애원로283',
 '충청북도충주시산척면샛강영길39',
 '충청남도논산시양촌면매죽헌로1665번길14-7',
 '경상남도창녕군대지면대지농공단지길40',
 '경기도파주시파주읍윗가마울길149',
 '경기도파주시파주읍윗가마울길149',
 '경상북도영덕군강구면소월1길16-10',
 '경상북도영덕군강구면소월1길16-10',
 '경상북도영덕군강구면소월1길16-10',
 '경상북도안동시풍산읍산업단지6길6',
 '경상북도안동시풍산읍산업단지6길6',
 '경상북도안동시풍산읍산업단지6길6',
 '서울특별시은평구증산로7길28-13',
 '서울특별시은평구증산로7길28-13',
 '경상북도안동시풍산읍㈜안동소주일품',
 '경상북도안동시풍산읍㈜안동소주일품',
 '강원속초시이목로104-49',
 '경기도용인시처인구양지면죽양대로2298-1',
 '전라북도남원시주천면장백산로130']

In [41]:
addr2 = pd.DataFrame(addr2,columns = ['주소2'])
addr2

,주소2
0,서울특별시서초구강남대로27길7-9우곡빌딩6층
1,울산광역시울주군상북면향산동길50
2,전라남도담양군담양읍삼다리78
3,서울특별시서초구강남대로27길7-9우곡빌딩6층
4,경상북도영주시단산면좌석리277-1
5,세종특별자치시전의면가느실길127
6,전라북도무주군안성면장무로1375-7
7,충청북도영동군영동읍조현길30
8,제주특별자치도제주시애월읍애원로283
9,충청북도충주시산척면샛강영길39


In [47]:
CB2 = pd.concat([CB,addr2],axis=1,)
CB2.head()

,술 종류,술 이름,양조장 위치,도수,용량(ml),"가격 - 공식홈페이지 가격(1병 기준), 없으면 네이버 스토어 최저가",별점 - 점수별 개수 평균(점수*개수/전체개수),탄산,맛,향,리뷰 빈도수(리뷰개수 가장 적은걸로 개수 통일화 후 랜덤으로 뽑기),"선물용(디자인, 병모양, 이쁨, 패키지, 설날, 크리스마스 등)",주소2
0,막걸리,배상면주가 느린마을 막걸리,서울특별시 서초구 강남대로 27길 7-9 우곡빌딩 6층,6,1000,10000,4.8,O,"고소하다\n뽀얗다\n담백하다\n크림,우유(부드럽다)\n달맛(달다,달짝지근하다,달달하...",사과\n요구르트\n곡물\n과일,18923,"선물','추석','설날','크리스마스','집들이','기념','대접'",서울특별시서초구강남대로27길7-9우곡빌딩6층
1,막걸리,복순도가,울산광역시 울주군 상북면 향산동길 50,6.5,935,12000,4.8,O,산뜻하다\n스파클링(탄산)\n꿀떡꿀떡\n순하다\n달짝지근하다(달달)\n부드럽다\n고...,청포도\r\n누룩\r\n포도\r\n복숭아\r\n향기,27821,"부모님','가족','지인','친구','명절','추석'",울산광역시울주군상북면향산동길50
2,막걸리,죽향도가,전라남도 담양군 담양읍 삼다리 78,6,600,3800,4.8,X,부드럽다\r\n순하다\r\n굉장하다\r\n깔끔하다\r\n달콤하다\r\n달달\r\n중...,꿀향,5916,생일\r\n부모님\r\n가족\r\n친구\r\n명절\r\n추석,전라남도담양군담양읍삼다리78
3,과실주,배상면주가 빙탄복,서울특별시 서초구 강남대로 27길 7-9 우곡빌딩 6층,7,370,4000,4.8,o,탄산음료같다\n도수가높지않다\n뒷맛이 깔끔,복분자,7754,혼술,서울특별시서초구강남대로27길7-9우곡빌딩6층
4,과실주,산내들 소백산 스위트 와인,경상북도 영주시 단산면 좌석리 277-1,12,750,15500,4.76,x,부드러움\n목넘김\n,향긋하다,3541,"초보자용, 가격대비, 고급스러움(맛, 포장)",경상북도영주시단산면좌석리277-1


In [48]:
CB2.drop(['도수','용량(ml)','가격 - 공식홈페이지 가격(1병 기준), 없으면 네이버 스토어 최저가','별점 - 점수별 개수 평균(점수*개수/전체개수)','탄산','맛','향','리뷰 빈도수(리뷰개수 가장 적은걸로 개수 통일화 후 랜덤으로 뽑기)','선물용(디자인, 병모양, 이쁨, 패키지, 설날, 크리스마스 등)'], axis=1, inplace=True)
CB2

,술 종류,술 이름,양조장 위치,주소2
0,막걸리,배상면주가 느린마을 막걸리,서울특별시 서초구 강남대로 27길 7-9 우곡빌딩 6층,서울특별시서초구강남대로27길7-9우곡빌딩6층
1,막걸리,복순도가,울산광역시 울주군 상북면 향산동길 50,울산광역시울주군상북면향산동길50
2,막걸리,죽향도가,전라남도 담양군 담양읍 삼다리 78,전라남도담양군담양읍삼다리78
3,과실주,배상면주가 빙탄복,서울특별시 서초구 강남대로 27길 7-9 우곡빌딩 6층,서울특별시서초구강남대로27길7-9우곡빌딩6층
4,과실주,산내들 소백산 스위트 와인,경상북도 영주시 단산면 좌석리 277-1,경상북도영주시단산면좌석리277-1
5,과실주,금이산농원 복숭아 와인,세종특별자치시 전의면 가느실길 127,세종특별자치시전의면가느실길127
6,과실주,덕유양조 산머루주,전라북도 무주군 안성면 장무로 1375-7,전라북도무주군안성면장무로1375-7
7,과실주,컨츄리 와인 영동포도 와인,충청북도 영동군 영동읍 조현길 30,충청북도영동군영동읍조현길30
8,청주,오메기 맑은술,제주특별자치도 제주시 애월읍 애원로 283,제주특별자치도제주시애월읍애원로283
9,청주,고헌정 동학 1957 특선 13도,충청북도 충주시 산척면 샛강영길 39,충청북도충주시산척면샛강영길39


In [50]:
CB2.to_csv('전통주2.csv',encoding="utf-8-sig", index=False)

In [14]:
import folium
map_osm = folium.Map(location = [37.559722, 126.975278],  zoom_start = 16)
map_osm.save("map.html")

In [16]:
CB_geoData = pd.read_csv("data_3.csv")
CB_geoData = CB_geoData.rename(columns={'field1':'store','field3':'addr','field4':'tel','field':'addr2','_X':'y','_Y':'x'})
CB_geoData = CB_geoData.drop(index=0)
CB_geoData

,store,field2,addr,tel,field5,_GC_TYPE,y,x
1,신사점,월-금 07:00~22:00 | 토 09:00~22:00| 일/공휴일 09:00~2...,서울시 강남구 도산대로 126,02-548-2741,서울특별시강남구도산대로126,정,127.022411,37.516969
2,차병원점,평일 07:00~18:00 | 토요일 07:00~15:00 | 일.공휴일 휴무,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시강남구논현로566강남차병원1층,정,127.034824,37.506440
3,강남대로점,"월-금 06:30-22:00 | 토-일,공휴일 08:00-22:00",서울시 서초구 강남대로 369 1층,02-588-5778,서울특별시서초구강남대로3691층,유,127.018486,37.518106
4,청담에스점,월-금 07:00~22:00 | 토 08:00~22:00| 일/공휴일 08:00~2...,"서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호",02-548-6052,"서울특별시강남구압구정로461네이처포엠빌딩B108,109호",정,127.047233,37.525255
5,강남에스점,월-목 07:00-22:30 | 금 07:00-23:00 I 토 08:00-23:0...,서울시 서초구 서초 1306-3호,02-593-5095,서울특별시서초구서초1306-3호,정,127.026049,37.500308
...,...,...,...,...,...,...,...,...
238,동탄타임테라스점,월-일 10:30 ~ 22:00,경기도 화성시 동탄중앙로220 타임테라스A 블록 1층 110호-1호,031-613-7684,경기도화성시동탄중앙로220타임테라스A블록1층110호-1호,정,127.068676,37.205391
239,천안신부동점,"평일, 주말 08:00~21:00",충남 천안시 동남구 만남로 48,041-566-0718,충청남도천안시동남구만남로48,정,127.157380,36.818595
240,일산동구청점,평일 07:00~22:00ㅣ주말/공휴일 08:00~22:00,경기도 고양시 일산동구 정발산로 38,031-903-3989,경기도고양시일산동구정발산로38,정,126.773131,37.656715
241,상암SBS프리즘타워점,"평일 07:30 ~ 19:00ㅣ주말,공휴일 휴점",서울 마포구 상암산로 82 SBS 프리즘 타워 1층 로비,02-562-9605,서울특별시마포구상암산로82SBS프리즘타워1층로비,정,126.892774,37.579715


In [17]:
map_CB = folium.Map(location = [37.560284, 126.975334], zoom_start=15)


In [18]:
for i,store in CB_geoData.iterrows():
    folium.Marker(location=[store['x'],store['y']],popup=store['store'],icon=folium.Icon(color='red',icon='star')).add_to(map_CB)

In [19]:
map_CB

In [13]:
map_CB.save('map_CB.html')


In [14]:
import webbrowser
webbrowser.open('map_CB.html')

True